In [1]:
import pandas as pd
import numpy as np
import tqdm
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

pd.set_option("max_rows", 200)
pd.set_option("max_columns", 50)
tqdm.tqdm.pandas()
%matplotlib inline

In [2]:
df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle").sort_values(["user_id", "timestamp"]).reset_index(drop=True)

In [3]:
import sys
sys.path.append("../")
# df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")
df = df.sort_values(["user_id", "timestamp"])
# large_user_id = df["user_id"].value_counts()
# large_user_id = large_user_id[large_user_id > 1000] 
# df = df[df["user_id"].isin(large_user_id.index)]
# df = df[df["answered_correctly"] != -1]

In [4]:
df["answered_correctly"] = df["answered_correctly"].replace(-1, np.nan)

In [70]:
df = df.reset_index()

ValueError: cannot insert level_0, already exists

In [5]:
def f(series):
    return series.shift(1).cumsum().fillna(0) / np.arange(len(series))


In [131]:
df_oof = pd.read_csv("../output/ex_172/20201202080625/oof_train_0_lgbm.csv")

In [132]:
df_oof = df_oof.set_index("row_id")

In [133]:
df_oof = df_oof.reset_index()
df_oof.columns = ["index", "predict", "target"]

In [134]:
df_oof

,index,predict,target
0,45,0.440669,0.0
1,46,0.523389,1.0
2,47,0.384600,0.0
3,48,0.428092,0.0
4,49,0.692595,1.0
...,...,...,...
1825192,10033861,0.776445,1.0
1825193,10033862,0.757160,1.0
1825194,10033863,0.926326,1.0
1825195,10033864,0.379459,0.0


In [135]:
df2 = pd.merge(df, df_oof, how="inner")

In [136]:
df2

,level_0,index,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,question_id,bundle_id,correct_answer,part,tags,tag,type_of,predict,target
0,45,45,1453702,24600,1262,0,31,0,0.0,16000.0,True,1262,1262,1,2,143 71 81 29,-1,NaN,0.440669,0.0
1,46,46,1477283,24600,1348,0,32,3,1.0,19000.0,True,1348,1348,3,2,143 105 81 92,-1,NaN,0.523389,1.0
2,47,47,1502268,24600,10686,0,33,3,0.0,14000.0,True,10686,10686,0,2,62 137 142 38 29,-1,NaN,0.384600,0.0
3,48,48,1526285,24600,754,0,34,3,0.0,19000.0,True,754,754,1,2,137 88 38 81,-1,NaN,0.428092,0.0
4,49,49,1550831,24600,801,0,35,0,1.0,14000.0,True,801,801,0,2,143 20 38 29,-1,NaN,0.692595,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1825192,10033861,10033861,55863864,2147481750,9980,0,45,2,1.0,3000.0,True,9980,9980,2,5,79,-1,NaN,0.776445,1.0
1825193,10033862,10033862,55884826,2147481750,5528,0,46,2,1.0,28000.0,True,5528,5528,2,5,14,-1,NaN,0.757160,1.0
1825194,10033863,10033863,55910967,2147481750,5279,0,47,3,1.0,14000.0,True,5279,5279,3,5,177,-1,NaN,0.926326,1.0
1825195,10033864,10033864,55938555,2147481750,5222,0,48,1,0.0,20000.0,True,5222,5222,2,5,175,-1,NaN,0.379459,0.0


In [137]:
df2 = df2[df2["answered_correctly"].notnull()]

In [138]:
roc_auc_score(df2["target"].values, df2["predict"].values)

0.788539142234133

In [139]:
df2["tsdiff"] = df2["timestamp"] - df2.groupby("user_id")["timestamp"].shift(1)

In [153]:
idx = df2[(df2["tsdiff"] > 0) & (df2["tsdiff"] < 20000) & (df2["part"] == 7)].index

In [154]:
df2.iloc[idx]["predict"].mean()

0.22289278179326644

In [155]:
len(idx)

814

In [156]:
df2["predict"].iloc[idx] = 0.25

/home/owner/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [157]:
roc_auc_score(df2["target"].values, df2["predict"].values)

0.7885359648426751